# About the "bisection"
The program use the "bisection" algorithm find the first valid frame. The anther one just use the order algorithm to find the first valid-frame. Thereby, you can see the first one are much faster and can record more valid frame than the first one. The first one spend 4 minutes 21 seconds while the second one spend 6 minutes 41 seconds. You can do the experiment in this file.